In [2]:
pip install dash



     ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
     ---------------------------------------- 0.1/10.2 MB 7.5 MB/s eta 0:00:02
     - -------------------------------------- 0.5/10.2 MB 7.8 MB/s eta 0:00:02
     --- ------------------------------------ 0.9/10.2 MB 9.3 MB/s eta 0:00:02
     ---- ----------------------------------- 1.0/10.2 MB 8.3 MB/s eta 0:00:02
     ------ --------------------------------- 1.7/10.2 MB 9.8 MB/s eta 0:00:01
     -------- ------------------------------- 2.2/10.2 MB 10.8 MB/s eta 0:00:01
     ---------- ----------------------------- 2.6/10.2 MB 10.2 MB/s eta 0:00:01
     ------------ --------------------------- 3.1/10.2 MB 11.0 MB/s eta 0:00:01
     -------------- ------------------------- 3.7/10.2 MB 11.9 MB/s eta 0:00:01
     --------------- ------------------------ 4.0/10.2 MB 12.2 MB/s eta 0:00:01
     ----------------- ---------------------- 4.5/10.2 MB 12.0 MB/s eta 0:00:01
     ------------------- -------------------- 5.1/10

In [ ]:


# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                    ],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                ),

                                html.Br(),

                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def render_pie_chart(selected_site):
    if selected_site == 'ALL':
        labels = ['Success', 'Failure']
        values = [spacex_df['class'].sum(), len(spacex_df) - spacex_df['class'].sum()]
    else:
        site_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        labels = ['Success', 'Failure']
        values = [site_df[site_df['class'] == 1]['class'].count(), site_df[site_df['class'] == 0]['class'].count()]
    fig = px.pie(values=values, names=labels, title=f'Success/Failure Count for Launch Site: {selected_site}')
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])
def update_scatter_chart(entered_site, payload_range):
    filtered_df = spacex_df
    if entered_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]

    min_payload = payload_range[0]
    max_payload = payload_range[1]
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= min_payload) & (filtered_df['Payload Mass (kg)'] <= max_payload)]

    fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                     title='Payload vs. Outcome for Selected Site')
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server()
    
  

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2023 01:00:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:00:33] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_3m1683089831.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:00:33] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_3m1683089831.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:00:33] "GET /_dash-component-suites/dash/deps/react@16.v2_9_3m1683089831.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:00:33] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_9_3m1683089831.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:00:33] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_9_2m1683089831.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:00:33] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_11m1683089831.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:00:33] "GET /_dash-component-suites/d

127.0.0.1 - - [03/May/2023 01:18:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:18:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:18:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:18:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:19:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:19:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:19:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:19:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2023 01:19:56] "POST /_dash-update-component HTTP/1.1" 200 -
